In [ ]:
import sys
sys.path.append("../scripts")

import os, torch
from sklearn.model_selection import train_test_split
import pickle
import torch_geometric.transforms as T
import numpy as np
from torch_geometric.nn.models import Node2Vec
from torch_geometric.data import DataLoader
from torch_geometric.nn import MessagePassing
from torch_geometric.data import Data
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
GCNConv._orig_propagate = GCNConv.propagate
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from torch_geometric.explain import GNNExplainer, Explainer
from sklearn.metrics import r2_score, mean_absolute_error
from models import *
from tg_functions import *

dropout_p = 0.5
bins = [int(i) for i in os.getenv("BINS", "400 800 1300 2100 3000 3700 4700 7020 9660").split(' ')]
bins = torch.tensor(bins, device='cuda' if torch.cuda.is_available() else 'cpu')
random_seed = 100

torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)

graph_num = 26  # Replace with your graph number

model_name = 'silver-eon-120'

weight_prefix = 'best_accuracy'  # Replace with your weight prefix

if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"Using CUDA device: {torch.cuda.get_device_name(0)}", flush = True)
else:
    device = torch.device('cpu')
    print("Using CPU", flush = True)


with open(f'../data/graphs/{graph_num}/linegraph_tg.pkl', 'rb') as f:
    data = pickle.load(f)

data.edge_index = data.edge_index.contiguous()
data.x = data.x.contiguous()
data.y = data.y.contiguous()

# Load the model with the GCN class
model = torch.load(f'../data/graphs/{graph_num}/models/{model_name}.pt', map_location=device)
model = model.to(device)

model.load_state_dict(torch.load(f'../data/graphs/{graph_num}/models/{model_name}_{weight_prefix}.pt', map_location=device))

model


In [ ]:
data.edge_index = data.edge_index.contiguous()
data.x = data.x.contiguous()
data.y = data.y.contiguous()
print(data.x.shape, data.edge_index.shape, data.y.shape, flush = True)
data = stratified_split(data)

criterion = torch.nn.MSELoss()


In [ ]:
def evaluate(model, data, critetion):
    model.eval()
    data.x = data.x.to(device)
    data.edge_index = data.edge_index.to(device)
    data.val_mask = data.val_mask.to(device)
    data.y = data.y.to(device)
    mask = data.val_mask.squeeze() & (data.y > 0).squeeze()
    out = model(data.x, data.edge_index)
    #target = torch.bucketize(data.y[mask], bins).squeeze()
    target = data.y[mask].squeeze()
    loss = criterion(out[mask].squeeze(), target)
    r2 = r2_score(target.detach().cpu().numpy(), out[mask].detach().cpu().numpy())
    mae = mean_absolute_error(target.detach().cpu().numpy(), out[mask].detach().cpu().numpy())
    return r2, mae


In [ ]:
r2, mae = evaluate(model, data, criterion)


In [ ]:
print(f"R2: {r2:.4f}, MAE: {mae:.4f}", flush = True)
